In [ ]:
#  Copyright (c) 2023 Arm Limited. All rights reserved.
#  SPDX-License-Identifier: Apache-2.0
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.


# Pruning and clustering for Arm® Ethos™ NPU - overview
[Pruning](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras) (setting weights to zero) and [clustering](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example) (grouping weights together into clusters) are optimisation techniques that modify the weights of a Machine Learning model and in some cases allow significant speed-up of the inference execution, reduction of the memory footprint and reduction in the overall power consumption of the system. Assuming you can optimise your workload without loss in accuracy (more on that later in the blog) and you target an Arm® Ethos™ NPU, you should consider pruning and clustering your neural network before feeding it to the Vela compiler and deploying it on the Ethos-U hardware. 

## Prerequisites
This article requires familiarity with the [Vela compiler](https://review.mlplatform.org/plugins/gitiles/ml/ethos-u/ethos-u-vela) and experience in running neural networks on the [Ethos-U NPU](https://armkeil.blob.core.windows.net/developer/Files/pdf/arm-ethos-u-processor-series-brief-v2.pdf). You can get familiar with the Vela compiler from [this blog](https://community.arm.com/arm-community-blogs/b/ai-and-ml-blog/posts/vela-complier-deploy-your-nn-model-on-the-arm-ethos-u-micronpu), and [here](https://review.mlplatform.org/plugins/gitiles/ml/ethos-u/ml-embedded-evaluation-kit/+/HEAD/docs/use_cases/inference_runner.md) you can see how to run a neural network on the Ethos-U NPU.  

## Why prune and cluster a neural network ? 
The Ethos-U hardware has a dedicated weight decoder to process the model weights. At the same time, Vela arranges the weights into blocks and the blocks are then fed to the hardware weight decoder. As part of the block arrangement process, Vela compresses sequences of zero weights and clusters of weights. To avoid any doubt, Vela uses lossless compression and the bit accuracy between the TensorFlow Lite Reference kernels and the Vela optimised ML model is preserved. If the model you feed to the Vela compiler is optimised to have sequences of zero weights and/or clusters of the same weights, Vela will be able to compress these weights very efficiently. The good compression of Vela would result in lower number of memory accesses by the NPU at runtime, which would mean that the MAC engines are not waiting on memory accesses resulting in better overall performance. In other words, if you have a workload where the MAC engines are stalling on memory accesses, you should consider pruning and clustering your neural network before compiling it with Vela.  
![When to consider pruning and clustering?](resources/pruning_clustering.png)

## How to know if the bottleneck in a ML model is memory access? 
The easiest way to determine if your model is bottlenecked by memory accesses is to run the network on different MAC configurations of the Ethos-U and analyse the overall performance for the model. Imagine you run the neural network on Arm® Ethos™-U55 NPU with 32 MACs/cc, 64 MACs/cc, 128 MACs/cc and 256 MACs/cc variants and you observe only slight improvement in the number of NPU Active cycles with the higher MAC configurations. This means the high number of MACs does not result in significantly better performance, usually because the MAC engines are not fed fast enough with data. You can use the [Corstone-300 Fixed Virtual Platform](https://developer.arm.com/downloads/-/arm-ecosystem-fvps) or the [MPS3 FPGA board](https://www.arm.com/products/development-tools/development-boards/mps3) to do this analysis. Worth noting that some layers are well known to be memory-bound and for them you can skip this step. For example, Fully Connected is highly memory bound operation because every weight is read once only. It is impossible to buffer the weights in memory and reuse them for the computation. In comparison, in a convolution you usually have small filter sizes which means you can buffer all the convolution weights in memory and reuse them for the computation. If your model is composed entirely of Fully Connected layers, the workload will be memory bound and you should consider pruning and clustering of the workload.  

## Install dependencies 
You should ensure you have Python3 installed on your machine. Running the following commands will install the libraries necessary to run the code sample.

In [ ]:
!pip3 install --upgrade pip3
!pip3 install tensorflow==2.12.0
!pip3 install numpy==1.23.5

## Base model 
Next, let us define and train a simple model with 3 Fully Connected layers and examine the performance on the Ethos-U55-128 when running on Corstone-300 reference design on FPGA. We will then prune and cluster the model and evaluate number of memory accesses of the NPU, overall performance and the size of the tflite file we load on device. 

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot
from tensorflow import keras
import tempfile

mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Example model with 3 Fully Connected layers trained on the MNIST dataset
def define_example_nn():
  # First, we define the model
  input_shape=(28,28)
  model = tf.keras.Sequential(
      [
          tf.keras.Input(shape=input_shape),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128,name='fc1',activation='relu'),
          tf.keras.layers.Dense(64,name='fc2',activation='relu'),
          tf.keras.layers.Dense(10,activation='softmax')
        ]
  )
  model.summary()
  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  # We train the model over 4 epochs
  model.fit(
    train_images,
    train_labels,
    epochs=4,
    validation_split=0.1,
  )
  return model

To deploy the model on the Ethos-U, we will need to quantise the model to int8, so we need a post-training quantisation function. 
Pruning and clustering also affects the accuracay of the model, so we need to keep track of the accuracy of the base model as well as the accuracy of the pruned and clustered versions of the workload. 

In [ ]:
# Function to do post-training quantisation of the model
def PTQ(model,name):  
  def rep_dataset():
      for i in range(50):
          img = train_images[i].astype(np.float32)
          img = np.expand_dims(img,0)
          yield [img]

  converter_quant = tf.lite.TFLiteConverter.from_keras_model(model)
  converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]
  converter_quant.representative_dataset = rep_dataset
  converter_quant.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

  converter_quant.inference_input_type = tf.int8
  converter_quant.inference_output_type = tf.int8
  tflite_model = converter_quant.convert()
  open(name+".tflite", 'wb').write(tflite_model)
  evaluate_accuracy(name+".tflite")

# Function evaluating the accuracy of a model
def evaluate_accuracy(nn): 
  interpreter = tf.lite.Interpreter(model_path = nn)
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  print("Testing quantized model: ",nn)
  num_correct = 0
  for data, label in zip(test_images,test_labels):
    image = ((data*255)-128).astype(np.int8)
    interpreter.set_tensor(input_details[0]['index'], np.expand_dims(image,0))
    interpreter.invoke()
    outputs = interpreter.get_tensor(output_details[0]['index'])
    predicted = np.argmax(outputs)
    num_correct += np.sum(predicted==label)
  print(f"Accuracy on test set: {100 * num_correct / len(test_images)}%")


We are ready to call our function defining and training the model with 3 Fully-Connected layers, apply post-training quantisation and evaluate the performance of the model. 

In [ ]:

def main():
  base_model = define_example_nn() # Base trained model
  PTQ(base_model,"model")

if __name__ == '__main__':
  main()


### Performance of the baseline model on the Ethos-U

We obtain 97% accuracy on the test set after the post-training quantisation routine. When compiling the quantised model with Vela, we can add the `--verbose-weights` CLI option to obtain information about the encoding of the weights. For the baseline model compiled in Shared_Sram memory mode, we obtain: 
```
Original Weights Size                          106.62 KiB 
NPU Encoded Weights Size                        93.17 KiB 
```
The Original Weight Size is 106KB and the NPU Encoded Weight Size is 93KB. During inference, the NPU will fetch the weights encoded by Vela from the AXI1 port of the NPU. Therefore, we will monitor closely how the NPU Encoded Weight Size changes after we apply pruning and clustering.  

When deploying on the Ethos-U, we place the model weights in the Flash and we obtain a total 91k NPU Active cycles for the inference on the Corstone-300 reference design. We also obtain 12k beats read on the AXI1 interface given by the ETHOSU_PMU_AXI1_RD_DATA_BEAT_RECEIVED PMU counter. The model size we load on-device is 104KB.  

## Pruned version of the base model
Next, let us prune the model. We will set 50% of the weights randomly to 0, will retrain for 2 epochs and will have 80% of the weights set to 0 at the end of the training.  To do that, we need to add a function to our code for pruning the model. This function will look like that: 

In [ ]:
# Function to prune the base model
def prune_model(model):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Compute end step to finish pruning after 2 epochs.
  batch_size = 128
  epochs = 2
  validation_split = 0.1 # 10% of training set will be used for validation set. 

  num_images = train_images.shape[0] * (1 - validation_split)
  end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

  # Define model for pruning.
  pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                final_sparsity=0.80,
                                                                begin_step=0,
                                                                end_step=end_step)
  }

  model_for_pruning = prune_low_magnitude(model, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  model_for_pruning.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  model_for_pruning.summary()
  logdir = tempfile.mkdtemp()

  callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
  ]

  model_for_pruning.fit(train_images, train_labels,
                    batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                    callbacks=callbacks)
  _, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

  print('Pruned test accuracy:', model_for_pruning_accuracy)
  print("Do post-train quant on the pruned model")
  model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
  return model_for_export


And we need to modify the `main` function slightly to pass on a copy of the base model for pruning. The main looks like that:

In [ ]:
def main():
  base_model = define_example_nn() # Base trained model
  base_model1 = tf.keras.models.clone_model(base_model) # Clone of trained model
  base_model1.set_weights(base_model.get_weights()) # Set the weights of the cloned model
  model_pruned = prune_model(base_model1) # Prune the cloned version of the base model
  PTQ(base_model,"model")
  PTQ(model_pruned,"model_pruned")
if __name__ == '__main__':
  main()

### Performance of the pruned model
After the post-training quantisation, the accuracy of the base model and the pruned model remains identical – 97%. You may even get a slight increase in accuracy of the pruned workload since you have added 2 more epochs to the training. How is it possible to have 80% of the weights set to 0 without losing accuracy? During training of the base model, it is common to have a lot of weights that are very close to 0, but are non-null. In the neural network design phase, we have not optimised the model architecture for embedded devices, we are just looking for correlations on the training data. As a result, there will be a lot of redundancy in the network. Pruning exploits that and allows us gets rid of some of this redundancy by setting intelligently some weights to 0 while preserving the accuracy. You may be wondering why aim for 80% sparsity by the end of the training and not another value. In the design of the neural network, you can experiment with different sparsity levels and examine the impact on the model accuracy. Empirical research has shown that on fully connected layers with big weight matrices like what we have in our model, you can apply high sparsity levels of 80% and even 90% without affecting the accuracy of the neural network. From the standpoint of the Ethos-U, the higher the sparsity, the better the performance will be.

For the encoding of the weights, Vela is reporting:  

```
Original Weights Size                          106.62 KiB 
NPU Encoded Weights Size                        30.12 KiB 
```

The Original Weight size has remained 106KB – this is logical because even though we have the same number of weights as in the first model, a lot of the weights are set to zero. However, the NPU Encoded Weight Size has reduced dramatically to 30KB. The is because the Vela compiler encodes the 0 weights very efficiently.  
 
At runtime, we obtain 33k Active NPU cycles on the Ethos-U55-128 and 4k beats of data read on AXI1. The size of the Vela optimised model is 40KB.

## Clustered version of the base model
Now, let us cluster the baseline model with 32 clusters. We will need the following function to apply clustering to the model weights. 

In [ ]:
def cluster_model(model,keep_sparsity):
  cluster_weights = tfmot.clustering.keras.cluster_weights
  CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

  clustering_params = {
    'number_of_clusters': 32,
    'cluster_centroids_init': CentroidInitialization.LINEAR,
    'preserve_sparsity': keep_sparsity
      }

  # Cluster a whole model
  clustered_model = cluster_weights(model, **clustering_params)

  # Use smaller learning rate for fine-tuning clustered model
  opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

  clustered_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=opt,
    metrics=['accuracy'])

  clustered_model.summary()
  # Fine-tune model
  clustered_model.fit(
    train_images,
    train_labels,
    batch_size=500,
    epochs=1,
    validation_split=0.1)
  _, clustered_model_accuracy = clustered_model.evaluate(test_images, test_labels, verbose=0)
  print('Clustered test accuracy:', clustered_model_accuracy)
  final_model = tfmot.clustering.keras.strip_clustering(clustered_model)
  return final_model


And the corresponding `main` looks like that:

In [ ]:
def main():
  base_model = define_example_nn() # Base trained model
  base_model2 = tf.keras.models.clone_model(base_model) # Clone of trained model
  base_model2.set_weights(base_model.get_weights()) 
  model_clustered = cluster_model(base_model2,False) # Cluster the clone of the base model
  PTQ(base_model,"model")
  PTQ(model_clustered,"model_clustered")
if __name__ == '__main__':
  main()

### Performance of the clustered model

Note that as we start from the baseline model, the neural network contains a lot of weights that are not equal to 0 yet(we haven't applied clustering to the pruned model yet). Hence, Vela will be able to encode the clusters of weights, but it won’t be able to encode sequences of 0 weights. Again, we keep the accuracy of 97% on the MNIST dataset.  Why did we select 32 clusters? The Ethos-U hardware can work with different number of clusters and the lower the number of clusters, the better the encoding by the compiler will be. The quantised model will have weights in int8(you can think of them as 256 clusters of weights), so it makes sense to be clustering with less than 256 clusters. If we have selected 16 clusters instead of 32, we would have obtained better encoding by the complier resulting in better overall performance at runtime. The optimal number of clusters is highly workload dependant, and with this blog we aim to give an intuition for the uplift during inference of pruning and clustering. You should experiment with different number of clusters for your specific model to understand what is the lowest number of clusters that you can afford without degrading the accuracy.  

In terms of the weight encoding, this time Vela reports:  
```
Original Weights Size                          106.62 KiB 
NPU Encoded Weights Size                        54.86 KiB 
```

The Original Weight Size remains the same, but the NPU Encoded Weight Size is 54KB. When pruning the model, we set 80% of the weights to 0 which resulted in better encoding than clustering with 32 clusters.  

At runtime, we obtain 56k NPU Active cycles and 7.1k beats read from the AXI1 interface. The size of the Vela optimised model we write to memory is 64KB.  

## Prune and cluster the base model
Lastly, we will apply pruning and clustering at the same time. We have all the necessary functions for that, and the final version of the `main` function looks like that:

In [ ]:
def main():
  base_model = define_example_nn() # Base trained model
  base_model1 = tf.keras.models.clone_model(base_model) # Clone of trained model
  base_model2 = tf.keras.models.clone_model(base_model) # Clone of trained model
  base_model1.set_weights(base_model.get_weights())
  base_model2.set_weights(base_model.get_weights())
  model_pruned = prune_model(base_model1)
  model_pruned_copy = tf.keras.models.clone_model(model_pruned) # Clone the pruned model
  model_pruned_copy.set_weights(model_pruned.get_weights())
  model_clustered = cluster_model(base_model2,False) # cluster without preserving sparsity
  model_pruned_clustered = cluster_model(model_pruned_copy,True) # cluster preserving the sparsity
  PTQ(base_model,"model")
  PTQ(model_pruned,"model_pruned")
  PTQ(model_clustered,"model_clustered")
  PTQ(model_pruned_clustered,"model_pruned_clustered")
if __name__ == '__main__':
  main()

### Performance of the pruned and clustered model
In terms of weight compression, Vela is reporting:  

```
Original Weights Size                          106.62 KiB 
NPU Encoded Weights Size                        22.22 KiB 
```

In this case, we benefit from the encoding of 0 weights and encoding of the 32 clusters which results in the lowest size of NPU Encoded Weights. This will translate to lowest number of NPU Active cycles. 

The accuracy of the model remains 97%. We obtain 26k NPU Active cycles, 3k beats read on AXI1 and the model size is 32KB. 

## Analysing the results
Let us put these results in a tabular form and draw a few conclusions:  

| Model we compile with Vela                   |NPU_ACTIVE cycles | AXI1 DATA BEAT_RECEIVED | Size of the _vela.tflite file |
| ----------------------------------------     |----------------- | ----------------------- | ----------------------------- |
| model.tflite(no pruning, no clustering)      |         91k      |           12k           |              104 KB           | 
| model_pruned.tflite(80% sparsity, no clustering)      |         34k      |           4k           |              40 KB           | 
| model_clustered.tflite(no pruning, 32 clusters)      |         56k      |           7k           |              64 KB           | 
| model_pruned_clustered.tflite(80% sparsity, 32 clusters)      |         26k      |           3k           |              32 KB           |

The model we defined is composed entirely of Fully Connected layers whose performance is limited by memory accesses. From the four workloads, the baseline model without pruning or clustering has the worst performance. On the pruned model, we get over 2.5x reduction in the number of NPU active cycles. We also obtain 3x reduction in the amount of memory traffic which will result in a significant power saving for the system. Furthermore, when the zero weights are compressed by Vela, the size of the model you store in memory reduces from 104 KB to 40 KB. On Fully Connected layers, pruning results in better compression compared to clustering since there are a lot of weights that can be made equal to zero, and the absolute best performance is obtained when combining pruning and custering. There was no loss in accuracy as we pruned and clustered the model, and this exploits the redundancy in the neural network. Finding the best balance between pruning and/or clustering is highly workload dependant. Recurrent Neural Networks such as LSTMs and GRUs also contain large Fully Connected layers – for such architectures we also recommend pruning and clustering before compiling the model with Vela.

 
# Conclusion
In summary, we demonstrated that if you have a memory bound model that you deploy on the Ethos-U, you can obtain better overall performance and lower number of memory accesses by pruning and clustering your neural network. Note this is true for any neural network architecture where the bottleneck is access to memory, not only for models composed of Fully Connected layers. If you have a convolutional neural network where the MAC engines are stalling on memory accesses, pruning and clustering will also result in improvement to the overall performance and power of the system.  